In [29]:
%store -r
%store

Stored variables and their in-db values:
base_model_uri                         -> 's3://jumpstart-cache-prod-ap-northeast-2/pytorch-
bucket                                 -> 'sagemaker-ap-northeast-2-242201274000'
controller_name                        -> 'V2LC'
default_bucket                         -> 'sagemaker-ap-northeast-2-242201274000'
image_uri                              -> '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-242201274000/cat-br
main_test_name                         -> 'HEV_P2_ACOverLoad_IG1_1'
model_image_uri                        -> '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com
model_package_group_name               -> 'CatBreedModelPackage'
preprocessing_code                     -> 'src/ad_parquet_preprocessing.py'
project_prefix                         -> 'Cat-Breed-Demo'
role                                   -> 'arn:aws:iam::242201274000:role/service-role/Amazo
sub_test_name

In [30]:
from sagemaker.workflow.parameters import ParameterString, ParameterInteger

evaluation_instance_type = ParameterString(
    name="EvaluationInstanceType",
    default_value="ml.m5.large"
)

evaluation_instance_count = ParameterInteger(
    name="EvaluationInstanceCount",
    default_value=1
)

In [31]:
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

script_eval = ScriptProcessor(
    image_uri=model_image_uri,
    command=["python3"],
    instance_type=evaluation_instance_type,
    instance_count=evaluation_instance_count,
    base_job_name="script-cat-breed-eval",
    role=role
)

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path = "evaluation.json"
)

In [32]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_eval = ProcessingStep(
    name="CatBreedEval",
    processor=script_eval,
    inputs=[ProcessingInput(source=train_model_artifact, destination='/opt/ml/processing/model'),
           ProcessingInput(source=test_preproc_dir_artifact, destination="/opt/ml/processing/test")],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")],
    code="cat-breed/evaluation.py",
    property_files=[evaluation_report]
)

In [33]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = "Cat-Breed-Demo"

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        evaluation_instance_type, 
        evaluation_instance_count,
    ],
    steps=[step_eval],
)

In [34]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'EvaluationInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.large'},
  {'Name': 'EvaluationInstanceCount', 'Type': 'Integer', 'DefaultValue': 1}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'CatBreedEval',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.EvaluationInstanceType'},
      'InstanceCount': {'Get': 'Parameters.EvaluationInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training:1.10.0-cpu-py38',
     'ContainerEntrypoint': ['python3',
      '/opt/ml/processing/input/code/evaluation.py']},
    'RoleArn': 'arn:aws:iam::242201274000:role/service-role/AmazonSageMaker-ExecutionRole-20240923T230631',
    'ProcessingInputs':

In [35]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [36]:
for idx, (key, item) in enumerate(execution.describe().items()):
    print(f"{idx}. key: {key}\n   value: \n{item}\n\n")

0. key: PipelineArn
   value: 
arn:aws:sagemaker:ap-northeast-2:242201274000:pipeline/Cat-Breed-Demo


1. key: PipelineExecutionArn
   value: 
arn:aws:sagemaker:ap-northeast-2:242201274000:pipeline/Cat-Breed-Demo/execution/yv706mxnrkmf


2. key: PipelineExecutionDisplayName
   value: 
execution-1731907695909


3. key: PipelineExecutionStatus
   value: 
Executing


4. key: CreationTime
   value: 
2024-11-18 05:28:15.845000+00:00


5. key: LastModifiedTime
   value: 
2024-11-18 05:28:15.845000+00:00


6. key: CreatedBy
   value: 
{'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-2:242201274000:user-profile/d-l0dltcg6j4kj/default-20240923T230629', 'UserProfileName': 'default-20240923T230629', 'DomainId': 'd-l0dltcg6j4kj', 'IamIdentity': {'Arn': 'arn:aws:sts::242201274000:assumed-role/AmazonSageMaker-ExecutionRole-20240923T230631/SageMaker', 'PrincipalId': 'AROATQZCSE2IHHIBWPD6G:SageMaker'}}


7. key: LastModifiedBy
   value: 
{'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-2:242201274

In [37]:
execution.wait()

In [38]:
execution.list_steps()

[{'StepName': 'CatBreedEval',
  'StartTime': datetime.datetime(2024, 11, 18, 5, 28, 16, 945000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 11, 18, 5, 30, 49, 536000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:242201274000:processing-job/pipelines-yv706mxnrkmf-CatBreedEval-kMVDhKekxC'}},
  'AttemptCount': 1}]

In [44]:
def get_proc_artifact(execution, client, kind=0):
    '''
    kind: 0 --> train
    kind: 1 --> test
    '''
    response = execution.list_steps()

    proc_arn = response[-1]['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
    proc_job_name = proc_arn.split('/')[-1]
    print(proc_job_name)
    
    response = client.describe_processing_job(ProcessingJobName = proc_job_name)
    test_preprocessed_file = response['ProcessingOutputConfig']['Outputs'][kind]['S3Output']['S3Uri'] 
    print(test_preprocessed_file)
    
    return test_preprocessed_file

import boto3
client = boto3.client("sagemaker")

eval_dir_artifact = get_proc_artifact(execution, client, kind=0 )

pipelines-yv706mxnrkmf-CatBreedEval-kMVDhKekxC
s3://sagemaker-ap-northeast-2-242201274000/Cat-Breed-Demo/yv706mxnrkmf/CatBreedEval/output/evaluation


In [46]:
%store eval_dir_artifact

Stored 'eval_dir_artifact' (str)
